## Importação dos módulos necessários

In [1]:
# usada para manipulação de arquivos de sistema operacional
import os

# importando conexão com banco de dados pyodbc
import pyodbc

# importando pandas
import pandas as pd

# alterando número de linhas
pd.set_option('display.max_rows', 300)

# alterando número de colunas
pd.set_option('display.max_columns', 70)

#  Módulo para leitura de arquivos em excel
import openpyxl

# modulo pára leitura de arquivos excel
import xlrd

# importando datetime
from datetime import date, timedelta


## Constantes e dicionários:

In [2]:

tipos_dados = {
    'data_hora' : ['data', 'movimento', 'duracao', 'duracao_1', 'duracao_2', 'duracao_3', 'duracao_4', 'duracao_5'],
    'inteiro' : ['codigo', 'quantidade', 'quantidade_1', 'quantidade_2', 'quantidade_3', 'quantidade_4', 'quantidade_5', 'quantidade_6', 'quantidade_7', 'quantidade_8'],
    'decimal' : ['percorrido', 'pontos_6', 'pontos_perdidos'],
    'texto' : []
}

## Diretório dos arquivos

In [3]:
# Diretorio de base
diretorio_base = 'G:/Meu Drive/TARUMA/PRONTUARIO_CONDUTOR/'

# Bancos de dados
db_prontuario = diretorio_base + 'SISTEMA_DE_GERENCIAMENTO_DE_CONDUTORES.accdb'

# relatorios de importacao 1: ranking do motorista 2: dados_motoristas
# Relatorio de importacao
relatorio_importacao = diretorio_base + 'DATABASE/RANK_.xls'

# Dados dos motoristas
dados_motoristas = diretorio_base + '/DATABASE/MOT_.xls'

## plano de ação
db_plano_acao = 'T:\LOGISTICA\PROJETOS\PLANO_ACAO_NOVO\PLANO_DE_ACAO.accdb'

## Importando dados do prontuário:

In [4]:
# execução de queries no banco de dados
def executa_sql(sql: str, banco_de_dados = db_prontuario ):

    # inicio a conexão
    connection = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + banco_de_dados + ';')

    #  Gero o curso
    cursor = connection.cursor()

    #  executo a query
    cursor.execute(sql)

    #  faço o commit da transação

    connection.commit()

    # fecho o cursor
    cursor.close()

    # fecho ao conexao
    connection.close()


def conexao_banco_de_dados(banco_de_dados = db_prontuario):

    # inicio a conexão
    return  pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + banco_de_dados + ';')


In [5]:
con = conexao_banco_de_dados()
cursor = con.cursor()
cursor.close()
con.close()

In [6]:
def buscar_cpf_motorista(parte_do_nome: str) -> str:
    return motoristas[motoristas['nome'].\
                      str.lower().\
                      str.contains(parte_do_nome)]

In [7]:
# lista de caracteres para serem alterados
special_characters = {'á': 'a',
                       'à': 'a',
                       'é': 'e',
                       'è': 'e',
                       'í': 'i',
                       'ì': 'i',
                       'ó': 'o',
                       'ò': 'o',
                       'ú': 'u',
                       'ù': 'u'}


def swap_special_characters(column):
    for char in column:
        if char in special_characters:
            column = column.replace(char, special_characters[char])
    return column


def normalize_columns(columns):

    return columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ', '_')


In [8]:
def adicionar_chave(df: pd.DataFrame)->pd.DataFrame:
    # fazendo uma cópia
    df2 = df.copy()
    
    # cpf
    cpf = df2['cpf']
    
    # codigo do motorista
    codigo = df2['codigo_do_motorista'].astype(str)
    
    # data
    data = df2['data'].astype(str)
    
    # distancia percorrida
    percorrido = df2['percorrido'].astype(str)
    
    # adicionando chave unica
    df2['unique_key'] = cpf + '_' + codigo + '_' + data + '_' + percorrido
    
    # retornando dataframe
    return df2

In [9]:
# Remover curva brusca
def remove_curva_brusca():
    sql = "UPDATE TB_TELEMETRIA SET CURVA_BRUSCA_QUANTIDADE=0"
    executa_sql(sql)

In [10]:
# limpar excesso de velocidade
def limpar_excessos_velocidade(data_inicial: str, data_final: str, cpf_motorista: str) -> None:
    sql = f"""
        UPDATE TB_TELEMETRIA 
            SET 
                EXCESSO_VELOCIDADE_QUANTIDADE=0,
                CURVA_BRUSCA_QUANTIDADE = 0,
                FRENAGEM_QUANTIDADE = 0,
                EXCESSO_VELOCIDADE_OCORRENCIA = 0
            WHERE DATA BETWEEN #{data_inicial}# and #{data_final}#
            AND CPF_FUNCIONARIO = '{cpf_motorista}'
        """
    print(sql)
    executa_sql(sql)

## Dados da telemetria

In [11]:
# importando arquivo excel
ranking_condutor =pd.read_excel(relatorio_importacao, skiprows=1, engine='xlrd')

# normalizando todos caracteres dos nomes das colunas
ranking_condutor.columns = normalize_columns(ranking_condutor.columns)

# Removendo caracteres inválidos
ranking_condutor.columns = ranking_condutor.columns.str.replace('[^A-Za-z0-9]+', '_', regex=True )

# convertendo data para datetime
ranking_condutor.data = pd.to_datetime(ranking_condutor.data, dayfirst=True)


WARNING *** file size (256905) not 512 + multiple of sector size (512)


In [13]:
 # convertendo dados para decimal
for coluna in tipos_dados["decimal"]:
    try:
        ranking_condutor[coluna] = ranking_condutor[coluna].str.replace(",", ".")
        ranking_condutor[coluna] = ranking_condutor[coluna].astype('float')
        
    except:
        
        print(f"Não foi possível converter {coluna}  em texto para em seguida converter em float")


    # Convertendo dados para data/hora
for coluna in tipos_dados["data_hora"]:
    try:
        ranking_condutor[coluna] = pd.to_datetime(ranking_condutor[coluna], dayfirst=True)
    
    except:
        print(f"Nâo foi possível converter {coluna}  em datetime")


 # Convertendo dados para inteiro
for coluna in tipos_dados["inteiro"]:
    try:
        
        ranking_condutor[coluna] = ranking_condutor[coluna].astype('int')
    except:
        print(f"Não foi possível converter {coluna} em inteiro")
        
        
print(ranking_condutor.dtypes)
        
ranking_condutor.head()

Não foi possível converter percorrido  em texto para em seguida converter em float
Não foi possível converter pontos_6  em texto para em seguida converter em float
Não foi possível converter pontos_perdidos  em texto para em seguida converter em float
Nâo foi possível converter movimento  em datetime
Nâo foi possível converter duracao  em datetime
Nâo foi possível converter duracao_5  em datetime
Não foi possível converter codigo em inteiro
nome                       object
codigo                    float64
departamento               object
data               datetime64[ns]
percorrido                float64
duracao                    object
movimento                  object
duracao_1          datetime64[ns]
pontos                      int64
duracao_2          datetime64[ns]
quantidade                  int32
pontos_1                    int64
quantidade_1                int32
pontos_2                    int64
quantidade_2                int32
pontos_3                    int64
quantidade_

,nome,codigo,departamento,data,percorrido,duracao,movimento,duracao_1,pontos,duracao_2,quantidade,pontos_1,quantidade_1,pontos_2,quantidade_2,pontos_3,quantidade_3,pontos_4,duracao_3,quantidade_4,pontos_5,quantidade_5,quantidade_6,duracao_4,quantidade_7,pontos_6,nota,qtd_ocorrencias,pontos_perdidos,quantidade_8,duracao_5
0,AILTON ALVES DE SIQUEIRA,96.0,ROTA JUIZ DE FORA,2023-05-25,114.485,03:38:26,03:38:26,2023-06-06 00:16:20,16,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"84,00",0,16.0,0,00:00:00
1,AILTON ALVES DE SIQUEIRA,96.0,ROTA JUIZ DE FORA,2023-05-26,105.010,03:48:36,03:21:06,2023-06-06 00:22:20,22,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:27:30,3,3.0,"75,00",0,25.0,0,00:00:00
2,AILTON ALVES DE SIQUEIRA,96.0,ROTA JUIZ DE FORA,2023-05-27,337.945,06:31:24,06:14:04,2023-06-06 01:32:40,92,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:17:20,1,1.0,"7,00",2,93.0,0,00:00:00
3,AILTON ALVES DE SIQUEIRA,96.0,ROTA JUIZ DE FORA,2023-05-30,165.265,04:30:03,04:30:03,2023-06-06 00:23:00,23,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"77,00",0,23.0,0,00:00:00
4,AILTON ALVES DE SIQUEIRA,96.0,ROTA JUIZ DE FORA,2023-05-31,306.424,06:14:15,06:14:15,2023-06-06 01:06:30,66,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"34,00",2,66.0,0,00:00:00


## Dados dos motoristas:

In [14]:
# Dados dos motoristas
motoristas = pd.read_excel(dados_motoristas, engine='xlrd')

# Normalizando os nomes nas colunas
motoristas.columns = normalize_columns(motoristas.columns)

# removendo caracteres no cpf
motoristas['cpf'] = motoristas.cpf.str.replace('.', '', regex=False).str.replace('-', '', regex=False)

# removendo dados inválidos nos nomes dos motoristas
motoristas.dropna(subset=['codigo_do_motorista'], inplace=True)

# colocando código do motorista como número inteiro
motoristas.codigo_do_motorista = motoristas.codigo_do_motorista.astype(int)

# Limpando  os dados inválidos para os códigos dos motoristas
ranking_condutor.dropna(subset=['codigo'], inplace=True)

# Convertendo para inteiro
ranking_condutor.codigo = ranking_condutor.codigo.astype(int)

## gerando dataset com o merge dos dados
dados = motoristas[['cpf', 'codigo_do_motorista']].merge(
    ranking_condutor,
    left_on='codigo_do_motorista',
    right_on='codigo',
    how= 'inner'
)

# Adicionando chave unica
dados = adicionar_chave(dados)


WARNING *** file size (67517) not 512 + multiple of sector size (512)


In [15]:
dados.head()

,cpf,codigo_do_motorista,nome,codigo,departamento,data,percorrido,duracao,movimento,duracao_1,pontos,duracao_2,quantidade,pontos_1,quantidade_1,pontos_2,quantidade_2,pontos_3,quantidade_3,pontos_4,duracao_3,quantidade_4,pontos_5,quantidade_5,quantidade_6,duracao_4,quantidade_7,pontos_6,nota,qtd_ocorrencias,pontos_perdidos,quantidade_8,duracao_5,unique_key
0,03957182603,96,AILTON ALVES DE SIQUEIRA,96,ROTA JUIZ DE FORA,2023-05-25,114.485,03:38:26,03:38:26,2023-06-06 00:16:20,16,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"84,00",0,16.0,0,00:00:00,03957182603_96_2023-05-25_114.485
1,03957182603,96,AILTON ALVES DE SIQUEIRA,96,ROTA JUIZ DE FORA,2023-05-26,105.010,03:48:36,03:21:06,2023-06-06 00:22:20,22,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:27:30,3,3.0,"75,00",0,25.0,0,00:00:00,03957182603_96_2023-05-26_105.01
2,03957182603,96,AILTON ALVES DE SIQUEIRA,96,ROTA JUIZ DE FORA,2023-05-27,337.945,06:31:24,06:14:04,2023-06-06 01:32:40,92,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:17:20,1,1.0,"7,00",2,93.0,0,00:00:00,03957182603_96_2023-05-27_337.945
3,03957182603,96,AILTON ALVES DE SIQUEIRA,96,ROTA JUIZ DE FORA,2023-05-30,165.265,04:30:03,04:30:03,2023-06-06 00:23:00,23,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"77,00",0,23.0,0,00:00:00,03957182603_96_2023-05-30_165.265
4,03957182603,96,AILTON ALVES DE SIQUEIRA,96,ROTA JUIZ DE FORA,2023-05-31,306.424,06:14:15,06:14:15,2023-06-06 01:06:30,66,2023-06-06,0,0,0,0,0,0,0,0,2023-06-06,0,0,0,0,2023-06-06 00:00:00,0,0.0,"34,00",2,66.0,0,00:00:00,03957182603_96_2023-05-31_306.424


## Inserção no banco de dados:

In [16]:
total_itens = dados.shape[0]
erros = []

for index, row in dados.iterrows():

    try:
        sql = f"""
        INSERT INTO TB_TELEMETRIA(
            COD_MOTORISTA,
            CPF_FUNCIONARIO,
            DEPARTAMENTO_MOTORISTA,
            DATA,
            CURVA_BRUSCA_QUANTIDADE,
            ACELERACAO_QUANTIDADE,
            FRENAGEM_QUANTIDADE,
            EXCESSO_VELOCIDADE_QUANTIDADE,
            EXCESSO_VELOCIDADE_OCORRENCIA,
            EXCESSO_VELOCIDADE_NAO_TOLERADO
        )
        values(
            {row.codigo},
            '{row.cpf}',
            '{row.departamento}',
            '{row.data}',
            {row.iloc[14]},
            {row.iloc[16]},
            {row.iloc[18]},
            {row.iloc[21]},
            {row.iloc[23]},
            {row.iloc[24]}
        )
        """
        executa_sql(sql)

    except:
        erros.append([ index, row.cpf, row.nome])


## Adicionando nomes que estão faltando no banco de dados

In [17]:
dados_erros = pd.DataFrame(erros)
dados_erros.head()

""


In [17]:
for index,row in  dados_erros[[1,2]].drop_duplicates().iterrows():
    try:
        sql = f""" INSERT INTO TB_FUNCIONARIO (CPF_FUNCIONARIO, DESC_FUNCIONARIO) VALUES('{row[1]}', '{row[2]}')"""
        executa_sql(sql)
    except:
        print('Não foi possível registrar os dados')

KeyError: "None of [Int64Index([1, 2], dtype='int64')] are in the [columns]"

## Atualização de registros e trabelas

In [27]:
buscar_cpf_motorista('andre alves')[['nome', 'cpf']]

,nome,cpf
8,ANDRE ALVES PEREIRA,78060702653


In [28]:
# Removendo curvas bruscas
remove_curva_brusca()

# Removendo excessos de velocidade justificados pelo motorista
limpar_excessos_velocidade(data_inicial='2021-05-29', data_final='2021-06-05', cpf_motorista='78060702653')


        UPDATE TB_TELEMETRIA 
            SET 
                EXCESSO_VELOCIDADE_QUANTIDADE=0,
                CURVA_BRUSCA_QUANTIDADE = 0,
                FRENAGEM_QUANTIDADE = 0,
                EXCESSO_VELOCIDADE_OCORRENCIA = 0
            WHERE DATA BETWEEN #2021-05-29# and #2021-06-05#
            AND CPF_FUNCIONARIO = '78060702653'
        


In [19]:
# sql = """
#     UPDATE TB_TELEMETRIA
#         SET
#             EXCESSO_VELOCIDADE_QUANTIDADE=0,
#             CURVA_BRUSCA_QUANTIDADE = 0,
#             FRENAGEM_QUANTIDADE = 0,
#             EXCESSO_VELOCIDADE_OCORRENCIA = 0
#         WHERE (month(data)) = 1
#     """

sql = """
    DELETE FROM TB_TELEMETRIA WHERE DATA >= #2023-05-29#

"""

executa_sql(sql)

## Visualizando Prontuário do Condutor

In [24]:
# Gerando prontuário
prontuario = pd.read_sql(sql="select * from qry_prontuario_condutor_classificado_divulgacao_online'", con=conexao_banco_de_dados()).fillna(0)

# Adicionando cpf mascarado ao prontuário do condutor
prontuario["cpf_m"] = [x[-4:] for x in prontuario['CPF_FUNCIONARIO']]

# alterando sequencia de colunas
prontuario = prontuario[[ 
    'cpf_m',
    'DEPTO',
    'ACELERA', 
    'CURVA',
    'FREIO', 
    'VELOCIDADE0',
    'VELOCIDADE1', 
    'VELOCIDADE2',
    'MULTAS', 
    'ADVERTENCIA',
    'AVARIAS', 
    'SOMA_PONTOS' ]
]

# Mudando nomes das colunas para facilitar a visualização dos motoristas
novos_nomes = {
     'DEPTO' : 'Setor',
     'FREIO' : 'Freadas Bruscas',
     'ACELERA' : 'Acelerações Bruscas',
     'CURVA' : 'Curvas bruscsa',
     'VELOCIDADE0' : 'Exc. 20%',
     'VELOCIDADE1' : 'Exc. 20%-30%',
     'VELOCIDADE2'  :'Exc.30%mais',
     'MULTAS' : 'Multas',
     'ADVERTENCIA' : 'Advertências',
     'AVARIAS' : 'Avarias',
     'SOMA_PONTOS' : 'Pontuação Geral',
     'cpf_m' : 'Cpf'   
    
}
prontuario.rename(columns=novos_nomes, inplace=True)

# Gerando excel do prontuário
prontuario.to_excel('C:/Users/Anderson/Documents/prontuario_do_condutor.xlsx', engine='openpyxl', index=False)

# Exibindo prontuário:
prontuario.sort_values(by='Pontuação Geral', ascending=False).head(200)

C:\Users\Anderson\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Cpf,Setor,Acelerações Bruscas,Curvas bruscsa,Freadas Bruscas,Exc. 20%,Exc. 20%-30%,Exc.30%mais,Multas,Advertências,Avarias,Pontuação Geral
21,8674,ROTA JUIZ DE FORA,0.0,0.0,0.0,65.0,7.0,3.0,0.0,3.0,0,331.0
24,9611,ROTA JUIZ DE FORA,0.0,0.0,2.0,13.0,0.0,0.0,0.0,9.0,0,52.0
38,8602,ROTA JUIZ DE FORA,0.0,0.0,2.0,16.0,0.0,0.0,0.0,0.0,0,52.0
6,7608,ROTA BARBACENA,1.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0,33.0
40,8635,ROTA JUIZ DE FORA,1.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0,30.0
41,1630,ROTA JUIZ DE FORA,1.0,0.0,0.0,1.0,0.0,1.0,0.0,6.0,0,30.0
54,2600,ROTA JUIZ DE FORA,0.0,0.0,0.0,2.0,1.0,0.0,0.0,12.0,0,28.0
29,2690,ROTA JUIZ DE FORA,0.0,0.0,0.0,6.0,0.0,0.0,0.0,9.0,0,27.0
52,8691,ROTA JUIZ DE FORA,1.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,0,24.0
57,2653,ROTA JUIZ DE FORA,0.0,0.0,2.0,5.0,0.0,0.0,0.0,3.0,0,22.0


In [25]:
# Exibindo prontuário:
prontuario[['Cpf','Pontuação Geral' ]].sort_values(by='Pontuação Geral', ascending=False).head(200)





,Cpf,Pontuação Geral
21,8674,331.0
24,9611,52.0
38,8602,52.0
6,7608,33.0
40,8635,30.0
41,1630,30.0
54,2600,28.0
29,2690,27.0
52,8691,24.0
57,2653,22.0


In [26]:

# Exibição do prontuário com nomes dos motoristas
pd.read_sql(sql="select * from qry_prontuario_condutor_classificado'", 
            con=conexao_banco_de_dados()).\
fillna(0).\
sort_values(by='SOMA_PONTOS', ascending = False).\
head(20)

C:\Users\Anderson\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,NOME,DEPTO,FREIO,ACELERA,CURVA,VELOCIDADE0,VELOCIDADE1,VELOCIDADE2,MULTAS,ADVERTENCIA,AVARIAS,SOMA_PONTOS
21,BRAULER EUSTAQUIO MENDES,ROTA JUIZ DE FORA,0.0,0.0,0.0,65.0,7.0,3.0,0.0,3.0,0,331.0
14,ALEX DA SILVA COSTA,ROTA JUIZ DE FORA,2.0,0.0,0.0,13.0,0.0,0.0,0.0,9.0,0,52.0
20,ANDRE LUIS DE JESUS BARRA,ROTA JUIZ DE FORA,2.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0,52.0
11,WESLLEY MATEUS DA SILVA,ROTA BARBACENA,0.0,1.0,0.0,11.0,0.0,0.0,0.0,0.0,0,33.0
31,FLAVIO JUNIOR RODRIGUES,ROTA JUIZ DE FORA,0.0,1.0,0.0,1.0,0.0,1.0,0.0,6.0,0,30.0
25,EDIMAR DA COSTA QUIRINO,ROTA JUIZ DE FORA,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,0,30.0
19,ANDRE LUIS BERNARDO,ROTA JUIZ DE FORA,0.0,0.0,0.0,2.0,1.0,0.0,0.0,12.0,0,28.0
47,MAURILIO DIAS DOS SANTOS JUNIOR,ROTA JUIZ DE FORA,0.0,0.0,0.0,6.0,0.0,0.0,0.0,9.0,0,27.0
34,JOSE BENEDITO DE SOUZA,ROTA JUIZ DE FORA,0.0,1.0,0.0,6.0,0.0,0.0,0.0,6.0,0,24.0
51,RODRIGO ALVES RENOVATO,ROTA JUIZ DE FORA,2.0,5.0,0.0,6.0,0.0,0.0,0.0,0.0,0,22.0


## Buscar Motoristas

In [ ]:
pd.read_sql(sql="select * from TB_TELEMETRIA WHERE CPF_FUNCIONARIO ='06066214667'", con=conexao_banco_de_dados()).fillna(0)

In [ ]:
for index,item in prontuario.sort_values(by='SOMA_PONTOS', ascending= False).iterrows():
    
    data_acao = date.today()
    data_prazo = data_acao + timedelta(7)
    nome = item.NOME
    pontos = item.SOMA_PONTOS
    excessos_velocidade_ate_20 = item.VELOCIDADE0
    excessos_velocidade_20_30 = item.VELOCIDADE1
    excessos_velocidade_30_mais = item.VELOCIDADE2
    curva_brusca = item.CURVA
    freada_brusca = item.FREIO
    aceleracao = item.ACELERA
    
    if item.SOMA_PONTOS >= 40:
              
        texto_acao = f"Advertir motorista {nome} por ter atingido a marca de {pontos} no prontuário do condutor."
        
        texto_comentario = f"Sua pontuação foi composta de: "\
        f"\t Excessos até 20%: {excessos_velocidade_ate_20}. Totalizando {excessos_velocidade_ate_20 * 3} pontos;"\
        f"\n\t Excessos entre 20% e 30%: {excessos_velocidade_20_30}. Totalizando {excessos_velocidade_20_30 * 10};"\
        f"\n\t Excessos acima de 30%: {excessos_velocidade_30_mais}. Totalizando {excessos_velocidade_30_mais * 21};"\
        f"\n\t Freadas bruscas: {freada_brusca}. Totalizando {freada_brusca * 2};"\
        f"\n\t Acelerações bruscas: {aceleracao}. Totalizando {aceleracao * 0};"\
        f"\n\t Curvas bruscas: {curva_brusca}. Totalizando {curva_brusca * 3};"\
        
        sql = f"""
            insert into tb_acao
            (
                data_acao,
                id_reuniao,
                id_origem,
                id_kpi,
                id_ferramenta,
                desc_acao,
                desc_comentario,
                prazo_acao,
                id_responsavel,
                id_status
            )
            
            values
            (
            '{data_acao}',
            {8},
            {4},
            {266},
            {1},
            '{texto_acao}',
            '{texto_comentario}',
            '{data_prazo}',
            {28},
            {1}
          )
        
        """
        executa_sql(sql, banco_de_dados = db_plano_acao)

In [ ]:
pontuacao = {
    40 : 'advertido por escrito com efeito verbal', 
    60 : 'advertido por escrito',
    80 : 'advertido por escrito',
    100 : 'advertido por escrito',
    120 : 'supenso de suas atividades por um dia para cumprir treinamento',
    140 : 'suspenso de suas atividades por dois dias',
    160 : 'desligado de suas atividades'
    
}

In [27]:
pd.read_sql(sql="select * from tb_acao", con=conexao_banco_de_dados(banco_de_dados = db_plano_acao)).fillna(0)

C:\Users\Anderson\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ID_ACAO,DATA_ACAO,ID_REUNIAO,ID_ORIGEM,ID_KPI,ID_FERRAMENTA,DESC_ACAO,DESC_COMENTARIO,PRAZO_ACAO,ID_RESPONSAVEL,ID_STATUS,DESC_FOLLOW,ID_RELATO,ID_SETOR
0,9805,2023-01-05,8,1,266,1,DESDOBRAR TREINAMENTO DO PRONTUÁRIO DO CONDUTO...,"PARA O TIME DA PUXADA, ENFATIZAR QUE EXCESSOS ...",2023-01-12,4,4,,0,1.0
1,9806,2023-01-05,13,1,273,1,Desenvolver ferramenta para conciliar a reposi...,Mapear o processo antes de iniciar as atividdes.,2023-01-12,4,2,Já está em fase de modelagem;,0,0.0
2,9807,2023-01-06,12,44,294,1,REALIZAR ENTREVISTA DE QUEBRAS E ADVERVIR INFO...,OCORRÊNCIA DE QUEBRA IMPACTOU NOSSO INDICADOR ...,2023-01-13,13,4,,0,0.0
3,9808,2023-01-08,12,1,265,1,Providenciar os devidos reparos no plano de aç...,Atualmente plano de ação não vêm funcionando c...,2023-01-15,4,4,Existe uma peculiaridade com o computador do j...,0,0.0
4,9809,2023-01-09,12,16,300,1,VERIFICAR SE PODEMOS DEVOLVER 1 PALETE DE SKOL...,,2023-01-16,3,1,,0,0.0
5,9810,2023-01-09,12,16,300,1,VERIFICAR SE PODEMOS DEVOLVER 120 FARDOS DE SK...,,2023-01-16,3,4,,0,0.0
6,9811,2023-01-10,5,4,227,1,"EDITAR DESLOCAMENTO DA ROTA MATIAS"" PARA QUE S...",ROTEIRIZADOR TEM MONTADO DESLOCAMENTO PELA EST...,2023-01-11,17,4,VOU EDITAR O DESLOCAMENTO ANTES DA EXPORTAÇÃO,0,0.0
7,9812,2023-01-11,5,4,285,1,ROTULAR E ALOCAR O CLIENTE 33028 KONNO EM ROTA...,CLIENTE COM ELEVADO TEMPO DE ATENDIMENTO QUE P...,2023-01-12,17,4,"CLIENTE ROTULADO, VAI SER ALOCADO EM EXTRA E F...",0,0.0
8,9813,2023-01-10,5,17,224,1,ACOMPANHAR E MONITORAR A ROTA DE SANTOS DUMONT...,CLIENTES EM AREA RURAL DISTANTE 15 KM DO CENTR...,2023-01-11,2,4,ROTA MONITORADA E CONCLUIDA COM SUCESSO.CARRO ...,0,0.0
9,9814,2023-01-11,5,4,227,1,EDITAR MAPA DO ROTEIRIZADOR PARA BLOQUEAR ESTR...,ROTAS TEM DADO DISPERSÃO POSITIVA POR CONTA DO...,2023-01-12,17,4,ALTERAÇÃO REALIZADA. AGUARDANDO APROVAÇÃO DA P...,0,0.0


In [ ]:
# lista de eventos
lista_eventos = 'data, cpf_funcionario, curva_brusca_quantidade'

# cpf do motorista
cpf_motorista = '07218640664'

# consulta
sql = f"select {lista_eventos} from tb_telemetria where cpf_funcionario = '{cpf_motorista}'"

# consulta
pd.read_sql(sql=sql, con=conexao_banco_de_dados()).fillna(0).query("curva_brusca_quantidade >= 2")